#### Building Autograde Engine

In [57]:
from random import Random
from math import sqrt

SEED = 5

random_gen = Random(x = SEED)

def gen_random (N=1):
    data_x, data_y = [], []
    for _ in range(N):
        data_x.append(random_gen.uniform(a=0, b=1))
    for _ in range(N):
        data_y.append(random_gen.uniform(a=0, b=1))
    return data_x, data_y
data_x, data_y = gen_random()
print(data_x, data_y)


[0.6229016948897019] [0.7417869892607294]


In [58]:
def loss(data_x, data_y, x_p = 0.3, y_p = 0.3):
    N = len(data_x)
    return (1/N) * sum([sqrt((x - x_p)**2 + (y - y_p)**2) for x,y in zip(data_x, data_y)])

print(loss(data_x, data_y))


0.5472122517293468


This is how to calculate $\frac{\partial \mathbb{L}}{\partial x_p}$:
<br><br>
$$
\mathbb{L} = \frac{1}{N} \sum_{i=0}^{N-1}[(x_{i} - x_{p})^{2} + (y_{i} - y_{p})^{2}]^{\frac{1}{2}}
\\
\mathbb{L} = C \sum_{i=0}^{N-1}{\mathbb{L}(x_i, y_i)}
\\
where\; C = \frac{1}{N}
$$


$$

\\
\frac{\partial \mathbb{L}}{\partial x_p} = -((x_i - x_p)^2 + (y_i - y_p)^2)^\frac{-1}{2} \;\; . \;\; (x_i - x_p) = \frac{\partial \mathbb{L}(x_i, y_i)}{\partial g_x}
$$

In [59]:
def calc_grad(data_x, data_y, x_p=0.3, y_p=0.3):
    grad_x, grad_y = 0. ,0.
    for x_i, y_i in zip(data_x, data_y):
        grad_x += ((((x_i-x_p)**2 + (y_i-y_p)**2)** -0.5) * (x_i-x_p))/len(data_x)
        grad_y += (((x_i-x_p)**2 + (y_i-y_p)**2)** -0.5) * (y_i-y_p) /len(data_y)
    return -grad_x, -grad_y
x_grad, y_grad = calc_grad(data_x, data_y)
print(x_grad, y_grad)


-0.5900849147094943 -0.8073411877467226


In [60]:
import torch

pnt = torch.tensor([0.3, 0.3])
pnt.requires_grad = True
pnt.retain_grad()
data_torch = torch.tensor([data_x, data_y])
data_torch = data_torch.t()

loss_torch = torch.mean(torch.sqrt(((data_torch-pnt)**2) . sum(dim = 1)))

loss_torch.backward()

print(loss_torch)
print(pnt.grad.data)

tensor(0.5472, grad_fn=<MeanBackward0>)
tensor([-0.5901, -0.8073])


## Building Autograd from scratch

In [61]:
class comp_node:
    def __init__(self, val, children = [], op = "assign"):
        self.val = val
        self.children = children
        self.grad = 0
        self.op = op
        self.backword_prop = lambda : None


    def __to_comp_node(self, obj):
        if not isinstance(obj, comp_node):
            return comp_node(val = obj)
        else:
            return obj
    def __sub__(self, other):
        
        other = self.__to_comp_node(other)   
        out = comp_node(val = (self.val - other.val),
                        children=[self, other], op ="subtraction")
        def _backword_pop():
            self.grad += out.grad * 1
            other.grad += out.grad * (-1)
        out.backword_prop = _backword_pop      
        return out
    

    def __rsub__(self, other):
        other = self.__to_comp_node(other)
        out = comp_node(val = (self.val - other.val),
                        children=[self, other],  op ="subtraction")
        return out
    
    def __pow__(self, exponent):
        if not isinstance(exponent, (int, float)):
            raise ValueError("Unsupported type")
        out = comp_node(val = self.val ** exponent,
                        children=[self], op = "power")
        def _backword_prop():
            self.grad +=out.grad * (exponent * self.val **(exponent - 1))
        out.backword_prop = _backword_prop
        return out


    def __eq__(self, other):
        return self.val == other.val
    
    def __add__(self, other):
        other = self.__to_comp_node(other)
        out = comp_node(val = self.val + other.val, 
                        children= [self, other],  op ="addition")
        def _backword_pop():
            self.grad += out.grad * 1
            other.grad += out.grad * 1
        out.backword_prop = _backword_pop
        return out
    def __radd__(self, other):
        other = self.__to_comp_node(other)
        out = comp_node(val = (self.val + other.val),
                        children=[self, other], op ="addition")
        return out

    def __mul__(self, other):
        other = self.__to_comp_node(other)
        out = comp_node(val = self.val * other.val, 
                        children=[self, other], op = "mult")
        def _back_prop():
            self.grad += out.grad * other.val
            other.grad += out.grad * self.val
        return out
    
    def __rmul__(self, other):
        other = self.__to_comp_node(other)
        return self * other
    def __repr__(self):
        return f"op:{self.op} | val:{self.val:.4f} | children:{len(self.children)} | grad: {self.grad}"


    
assert comp_node(val = 5). val == 5, "assignment falied"
assert (comp_node(val = 5) - comp_node(val =3 )).val == 2
assert(comp_node(val = 5) - 3 ).val ==2
assert(5 - comp_node(val = 3).val ==2)
assert (comp_node(val=5)**2).val == 25
assert(comp_node(val=5)**2) == comp_node(val =25)
assert (comp_node(val = 5) + comp_node(val =3 )).val == 8
assert(comp_node(val = 5) + 3 ).val ==8
assert(5 + comp_node(val = 3).val ==8)
assert (comp_node(val=5)*2).val == 10
assert(comp_node(val=5)*2) == comp_node(val =10)


In [65]:

x_p, y_p = comp_node(val=0.3), comp_node(val=0.3)

def loss_graph(x_p, y_p, data_x, data_y):
    # print(data_x, data_y)
    # print(f"x_p{x_p.val} | y_p{y_p}")
    I_x, I_y = x_p - data_x, y_p - data_y
    g_x, g_y = I_x**2, I_y **2
    M = g_x + g_y
    L = M **0.5
    return L, [L, M, g_x,g_y, I_x, I_y, x_p, y_p]

l, reverse_topo = loss_graph(x_p, y_p, data_x[0], data_y[0])
reverse_topo[0].grad = 1
for i, node in enumerate(reverse_topo):
    node.backword_prop()
    print(i, node)
    

0 op:power | val:0.5472 | children:1 | grad: 1
1 op:addition | val:0.2994 | children:2 | grad: 0.9137222319490423
2 op:power | val:0.1043 | children:1 | grad: 0.9137222319490423
3 op:power | val:0.1952 | children:1 | grad: 0.9137222319490423
4 op:subtraction | val:-0.3229 | children:2 | grad: -0.5900849147094943
5 op:subtraction | val:-0.4418 | children:2 | grad: -0.8073411877467226
6 op:assign | val:0.3000 | children:0 | grad: -0.5900849147094943
7 op:assign | val:0.3000 | children:0 | grad: -0.8073411877467226
